<h1><center>[Notebooks](../) - [Access to Geospatial data](../Access to Geospatial data)</center></h1>

This notebook is heavly based on the [Python GDAL/OGR Cookbook](https://pcjericks.github.io/py-gdalogr-cookbook/)

In this first example we'll learn how to generate a proper OGR geometry of type "Geometry Collection" which allow us to store multiple types of geometry (point, line, polygon) in a single vector file (note: this is a great advantage compared with the "standard but obsolete" ESRI Shape File data format, which is limited to only one type of geometry). 

Let's re-use the example from the notebook in  [Numerical Cartograph/The Geodesic Problem](../Numerical Cartography/The Geodesic Problem.ipynb), where using [geograhiclib]() we compute the shortest path (geodesic) from New York, (NY) to Delhi, (India):

In [ ]:
from geographiclib.geodesic import Geodesic

lat1,lon1 = (40.7143528, -74.0059731)  # New York, NY
lat2,lon2 = (1.359, 103.989)   # Delhi, India
g = Geodesic.WGS84.Inverse(lat1, lon1, lat2, lon2)
g

Now compute a list of points along the geodesic curve with a fixed distance of 100000m

In [ ]:
gc = [Geodesic.WGS84.Direct(lat1, lon1, g['azi1'], i) for i in range(0,int(g['s12']),100000)]

Now let's start with OGR! 

We will first create a geometry collection with:

In [ ]:
from osgeo import ogr
geomcol =  ogr.Geometry(ogr.wkbGeometryCollection)

let's add 2 point features for the 2 location we used to compute the geodesic curve:

In [ ]:
point1 = ogr.Geometry(ogr.wkbPoint)
point1.AddPoint(lon1,lat1)
geomcol.AddGeometry(point1)

point2 = ogr.Geometry(ogr.wkbPoint)
point2.AddPoint(lon2,lat2)
geomcol.AddGeometry(point2)

Let's add to our GeometryCollection a new line feature:

In [ ]:
line = ogr.Geometry(ogr.wkbLineString)

And let's populate the newly generated line feature looping through the list of coordinates of our geodesic path.

In [ ]:
[line.AddPoint(i['lon2'],i['lat2']) for i in gc]
geomcol.AddGeometry(line)

And finally export the data as GeoJson string

In [ ]:
data = geomcol.ExportToJson()

To save on file the new dataset we will ```echo``` the ```geojson``` string to a file

In [ ]:
!echo '{data}' > /tmp/geojson.geojson

The ```gist``` utility (not installed on the live) can be used to paste the content of a text file online as  a *gist*  for easy visualization on github

In [ ]:
#!gist -p /tmp/geojson.geojson

In [ ]:
from CesiumWidget import CesiumWidget

In [ ]:
cesiumExample = CesiumWidget(width="100%",geojson=data, enable_lighting=True)

In [ ]:
cesiumExample